In [1]:
!pip install transformers wikipedia neo4j kuzu langchain > /dev/null

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
import IPython
import pandas as pd

**LOADING DATA** *

In [19]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=592dafaf83fc9331e62e0af658549b75db6e2124cc1cc75ecbf1812969f9d34b
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9eaec5000e01337430d281c409a4c80df662878d9c45be63c641610cdb039409
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [20]:
from keybert import KeyBERT
import pandas as pd
import csv

In [21]:
df=pd.read_csv("/content/summarized00 (1).csv")

In [52]:
fdf=df[["subTitle","PMI_subtitle","summary"]]
fdf

,subTitle,PMI_subtitle,summary
0,NaN,NaN,<pad> True story: Project Risk Management aims...
1,NaN,NaN,<pad> True: Plan Risk Management is the proces...
2,11.1.1.1 Project charter,8.5.2 Add Corresponding Risk Responses to the ...,<pad><extra_id_0> project charter documents hi...
3,11.1.1.2 Project management plan,4.3.2 Templates,<pad> <unk>î<unk>®<unk> <unk>î<unk>®<unk> <unk...
4,11.1.1.3 Project documents,2.7 Responsibility for Project Risk Management,<pad> ct docum ents project documents consider...
...,...,...,...
85,11.7.3.1 Work performance information,5.2.7 Risks Linked to Project Objectives,<pad> <unk>î<unk>®<unk> <unk> <unk> <unk> <unk...
86,11.7.3.2 Change requests,8.5.2 Add Corresponding Risk Responses to the ...,<pad> e request described section monitor risk...
87,11.7.3.3 Project management plan updates,4.3.2 Templates,<pad> <unk>î<unk>®<unk> experience existing be...
88,11.7.3.4 Project documents updates,2.5 Iterative Process,<pad><extra_id_0> ed<extra_id_1> uurisk regist...


# Load the REBEL model

In [3]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
test_text = "This notebook shows how to use LLMs to provide a natural language interface to Kùzu database."

In [5]:
# Tokenizer text
model_inputs = tokenizer(test_text,
                          max_length=512,
                          padding=True,
                          truncation=True,
                        return_tensors='pt')

print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

# Generate
gen_kwargs = {
    "max_length": 216,
    "length_penalty": 0,
    "num_beams": 5,
    "num_return_sequences": 4
}
generated_tokens = model.generate(
    **model_inputs,
    **gen_kwargs,
)
decoded_preds = tokenizer.batch_decode(generated_tokens,
                                        skip_special_tokens=False)

decoded_preds

Num tokens: 23


['<s><triplet> LLMs <subj> natural language interface <obj> use</s>',
 '<s><triplet> LLMs <subj> natural language <obj> use</s><pad>',
 '<s><triplet> LLM <subj> natural language interface <obj> use</s>',
 '<s><triplet> LLMs <subj> natural language <obj> subclass of</s>']

In [45]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [46]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [47]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text,
                             max_length=512,
                             padding=True,
                             truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 216,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }
    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

In [48]:
small_kb = from_small_text_to_kb(test_text)

In [49]:
small_kb.print()

Relations:
  {'head': 'LLMs', 'type': 'use', 'tail': 'natural language interface'}
  {'head': 'LLMs', 'type': 'use', 'tail': 'natural language'}
  {'head': 'LLM', 'type': 'use', 'tail': 'natural language interface'}


In [11]:
text = """If you are just getting started,
and you have relatively simple apis,
you should get started with chains.
Chains are a sequence of predetermined steps,
so they are good to get started with as they
give you more control and let you understand
what is happening better."""

kb = from_small_text_to_kb(text,
                           verbose=True)
kb.print()

Num tokens: 62
Relations:
  {'head': 'Get started with chains', 'type': 'subclass of', 'tail': 'Chain'}
  {'head': 'Get started with chains', 'type': 'uses', 'tail': 'Chain'}
  {'head': 'Get started with chains', 'type': 'instance of', 'tail': 'Chain'}


# Split spans: from long text to KB

In [50]:
class SpankB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head",
                                                     "type",
                                                     "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
    def get_content(self):
        content = "Relations:\n"
        for r in self.relations:
            content += f"  {r}\n"
        return content

    def save_csv(self,file_name):
        print(f"Saving to file {file_name}")
        reln_df = pd.DataFrame(self.relations)
        reln_df.to_csv(file_name,index=False)

In [51]:
def from_text_to_kb(text,
                    span_length=50,
                    verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = SpankB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [44]:
def extract_relations_from_text(text):
    kb = from_text_to_kb(text, verbose=True)
    return kb

In [53]:
fdf['relations'] = fdf['summary'].apply(extract_relations_from_text)

Input has 78 tokens
Input has 2 spans
Span boundaries are [[0, 50], [28, 78]]
Input has 77 tokens
Input has 2 spans
Span boundaries are [[0, 50], [27, 77]]
Input has 104 tokens
Input has 3 spans
Span boundaries are [[0, 50], [27, 77], [54, 104]]
Input has 102 tokens
Input has 3 spans
Span boundaries are [[0, 50], [26, 76], [52, 102]]
Input has 96 tokens
Input has 2 spans
Span boundaries are [[0, 50], [46, 96]]
Input has 95 tokens
Input has 2 spans
Span boundaries are [[0, 50], [45, 95]]
Input has 86 tokens
Input has 2 spans
Span boundaries are [[0, 50], [36, 86]]
Input has 102 tokens
Input has 3 spans
Span boundaries are [[0, 50], [26, 76], [52, 102]]
Input has 223 tokens
Input has 5 spans
Span boundaries are [[0, 50], [43, 93], [86, 136], [129, 179], [172, 222]]
Input has 104 tokens
Input has 3 spans
Span boundaries are [[0, 50], [27, 77], [54, 104]]
Input has 81 tokens
Input has 2 spans
Span boundaries are [[0, 50], [31, 81]]
Input has 77 tokens
Input has 2 spans
Span boundaries are 

<ipython-input-53-f6d46a0ab7dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf['relations'] = fdf['summary'].apply(extract_relations_from_text)


In [54]:
for i, x in enumerate(fdf["relations"]):
    fdf["relations"].iloc[i] = x.get_content()

In [59]:
L=list()
for x in fdf["relations"]:
  L.append(x.get_content())

In [60]:
L

["Relations:\n  {'head': 'Risk Management', 'type': 'subclass of', 'tail': 'manage risks', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'Project Risk Management', 'type': 'facet of', 'tail': 'project management', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'Risk Management', 'type': 'subclass of', 'tail': 'manage risk', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'risk', 'type': 'has part', 'tail': 'reward', 'meta': {'spans': [[28, 78]]}}\n  {'head': 'risk', 'type': 'different from', 'tail': 'reward', 'meta': {'spans': [[28, 78]]}}\n  {'head': 'reward', 'type': 'different from', 'tail': 'risk', 'meta': {'spans': [[28, 78]]}}\n",
 "Relations:\n  {'head': 'Plan Risk Management', 'type': 'facet of', 'tail': 'risk management', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'Plan Risk Management', 'type': 'subclass of', 'tail': 'risk management', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'Plan Risk Management', 'type': 'use', 'tail': 'risk management', 'meta': {'spans': [[0, 50]]}}\n  {'head': 'outputs

In [63]:
import re
pattern = r"{'head': '(.*?)', 'type': '(.*?)', 'tail': '(.*?)', 'meta': {'spans': \[\[.*?\]\]}}"
data_list = []

for entry in L:
    matches = re.findall(pattern, entry)
    data_list.extend(matches)

# Define the CSV file path
csv_file = 'output1.csv'

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['head', 'type', 'tail', 'meta'])
    for item in data_list:
        writer.writerow(item)

In [30]:
file_path = '/content/output.csv'
fdf.to_csv(file_path, index=False)